### Handling multiple sequences

In [4]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=6, linewidth=75, sci_mode=False)

在前一节中，我们探讨了最简单的用例: 对一个小长度的单个序列进行推理。然而，一些问题已经出现了:
- 我们如何处理多个序列?
- 我们如何处理不同长度的多个序列?
- 词汇表索引是允许模型正常工作的唯一输入吗?
- 是否存在太长的序列?

让我们看看这些问题会带来什么样的问题，以及如何使用Transformers API来解决这些问题。

#### Models expect a batch of inputs

在前面的练习中，您看到了如何将序列转换为数字列表。让我们将这个数字列表转换为一个张量，并将其发送给模型:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

# 下面这行会报错
model(input_ids)

为什么失败了?“我们遵循了第2部分中管道的步骤。

问题在于向模型只传入了单个序列，而Transformers模型默认情况下是需要列表。在这里，当我们将标记器应用于序列时，我们试图在幕后完成它所做的一切。但如果你仔细观察，你会发现标记器不仅将输入id列表转换为tensor，它还在上面添加了一个维度:

In [9]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
tokenized_inputs, tokenized_inputs.input_ids.shape, tokenized_inputs.attention_mask.shape

({'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
          12172,  2607,  2026,  2878,  2166,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 torch.Size([1, 16]),
 torch.Size([1, 16]))

让我们再试一次，添加一个新的维度:

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012]])
Logits: tensor([[-2.727622,  2.878939]], grad_fn=<AddmmBackward0>)


批处理是将多个句子一次性发送到模型中的行为。如果你只有一个句子，你可以用单个序列构建一个批处理:

batched_ids = [ids, ids]

这是两个相同序列的一批!

批处理允许模型在输入多个句子时工作。使用多个序列就像用单个序列构建批处理一样简单。不过，还有第二个问题。当你试图将两个(或更多)句子批处理在一起时，它们可能有不同的长度。如果你以前用过tensor，你知道它们需要是矩形的，所以你不能直接把输入id列表转换成张量。为了解决这个问题，我们通常填充输入。

#### Padding the inputs

以下列表的列表不能转换为张量:

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

为了解决这个问题，我们将使用padding来使tensor具有矩形形状。padding通过在值较少的句子中添加一个称为padding的特殊单词来确保所有句子具有相同的长度。例如，如果你有10个句子，10个单词和1个句子，20个单词，填充将确保所有的句子都有20个单词。在我们的例子中，得到的张量是这样的:

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

padding token ID可以在tokenizer.pad_token_id中找到。让我们使用它，将我们的两个句子分别发送到模型中，并将它们一起批量发送:

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.569370, -1.389461]], grad_fn=<AddmmBackward0>)
tensor([[ 0.580311, -0.412535]], grad_fn=<AddmmBackward0>)
tensor([[ 1.569371, -1.389462],
        [ 1.337351, -1.216321]], grad_fn=<AddmmBackward0>)


我们批处理预测中的logits有问题:第二行应该与第二句的logits相同，但是我们得到了完全不同的值!

这是因为Transformer模型的关键特性是将每个令牌置于上下文中的注意层。这将考虑填充标记，因为它们涉及序列的所有标记。为了在通过模型传递不同长度的单个句子时获得相同的结果，或者在传递具有相同句子并应用了填充的批处理时获得相同的结果，我们需要告诉这些注意层忽略填充标记。这是通过使用attention mask来实现的。

#### Attention masks

Attention masks是与输入id张量形状完全相同的张量，由0和1填充:1表示需要注意对应的令牌，0表示不需要注意对应的令牌(即模型的注意层应该忽略它们)。

让我们用一个注意力掩码来完成前面的例子:

In [12]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.569371, -1.389462],
        [ 0.580311, -0.412536]], grad_fn=<AddmmBackward0>)


现在我们得到了批处理中第二个句子的相同对数。

请注意，第二个序列的最后一个值是一个padding ID，它在attention mask中是一个0值。

#### Longer sequences

对于Transformer模型，我们可以传递给模型的序列长度是有限制的。大多数模型处理多达512或1024个token序列，并且在被要求处理更长的序列时会崩溃。这个问题有两种解决方案:
- 使用具有更长的支持序列长度的模型
- 截断你的序列

模型支持不同的序列长度，有些模型专门处理非常长的序列。Longformer是一个例子，另一个例子是LED。如果您正在处理一个需要很长序列的任务，我们建议您查看这些模型。

否则，我们建议您通过指定max_sequence_length参数来截断序列:

sequence = sequence[:max_sequence_length]